# <b><center> Social Networks EDA </b></center>


## Entendimiento de los Datos:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## <u>Leer los datos que en el nombre contienen un mes:</u></br>
### <b>Instagram:</b>

In [4]:
ig_june= pd.read_csv('DATA/instagramjune2022.csv')
ig_june

,instagram name,influencer name,Category_1,Category_2,Subscribers count,Views avg.,Likes avg,Comments avg.
0,433,433,Sports with a ball,NaN,50.7M,Spain,274.9K,456.7K
1,____kimwoobin,김우빈,Lifestyle,NaN,3.6M,Indonesia,449.4K,592.7K
2,__youngbae__,TAEYANG,Music,NaN,13.4M,Indonesia,269.6K,304.1K
3,_agentgirl_,НАСТЯ ИВЛЕЕВА,Shows,NaN,18.6M,Russia,164.4K,189.4K
4,_hakkencoser_,Hakken 八犬,Lifestyle,Photography,3.5M,United States,525.1K,643K
...,...,...,...,...,...,...,...,...
1017,zidane,zidane,Sports with a ball,NaN,33M,Spain,412.4K,560.8K
1018,zidane,zidane,Sports with a ball,NaN,33M,Spain,412.4K,560.8K
1019,zkdlin,KAI,Music,NaN,14.1M,Indonesia,1.4M,1.7M
1020,zo,Lonzo Ball,Sports with a ball,NaN,14M,United States,145.8K,262K


In [6]:
ig_sep= pd.read_csv('DATA/instagramsep2022.csv')
ig_sep

,S.no,Instagram name,Name,Subscribers,Audience country,Authentic engagement\n,Engagement average\r\n,Category_1,Category_2
0,1,cristiano,Cristiano Ronaldo,477.9M,India,5M,6.2M,Sports with a ball,NaN
1,2,kyliejenner,Kylie 🤍,368.1M,United States,3.5M,5.5M,Fashion,Modeling
2,3,arianagrande,Ariana Grande,329.6M,United States,2.9M,4M,NaN,NaN
3,4,leomessi,Leo Messi,358.6M,Indonesia,2.7M,3.5M,Sports with a ball,Family
4,5,zendaya,Zendaya,151.1M,United States,4.3M,5.8M,Cinema & Actors/actresses,Fashion
...,...,...,...,...,...,...,...,...,...
1045,996,jadethirlwall,jade amelia thirlwall,9.4M,United States,174.2K,228.1K,NaN,NaN
1046,997,ninja,User Not Found,12.9M,United States,127.8K,163.9K,Computers & Gadgets,Machinery & Technologies
1047,998,myriamfares,Myriam Fares,20.6M,Iraq,75.8K,102.4K,NaN,NaN
1048,999,optimushwang,황민현,4.2M,South Korea,426K,508K,Lifestyle,NaN


In [5]:
ig_nov= pd.read_csv('DATA/instagramNov2022.csv')
ig_nov

,s.no,Name,Instagram Name,Category-1,Followers,\nCountry,Eng. (Auth.),Eng. (Avg.),Category-2
0,1,cristiano,Cristiano Ronaldo,Sports with a ball,483.1M,India,4.5M,5.6M,NaN
1,2,kyliejenner,Kylie 🤍,Fashion,370M,United States,2.8M,4.4M,Modeling
2,3,leomessi,Leo Messi,Sports with a ball,357.8M,Indonesia,3.2M,4.1M,Family
3,4,arianagrande,Ariana Grande,Music,331.6M,United States,2.9M,4M,NaN
4,5,zendaya,Zendaya,Cinema & Actors/actresses,152.2M,United States,4.8M,6.5M,Fashion
...,...,...,...,...,...,...,...,...,...
995,996,thenewclassic,Iggy Azalea,Music,16.7M,United States,82.1K,132.8K,NaN
996,997,min.nicha,MINNIE,Lifestyle,4.2M,Thailand,385.5K,526.6K,Music
997,998,francety,Francia James,Adult content,11.4M,India,149.9K,196.6K,NaN
998,999,silambarasantrofficial,Silambarasan TR,Shows,10.2M,India,169.6K,216.6K,Lifestyle


In [3]:
ig_dec= pd.read_csv('DATA/INSTAGRAMDEC2022.csv')
ig_dec

,Rank,name,instagram name,Category_1,Category_2,followers,country,Eng. (Auth.),Eng. (Avg.)
0,1,leomessi,Leo Messi,Sports with a ball,Family,409.8M,Argentina,17.8M,23.4M
1,2,cristiano,Cristiano Ronaldo,Sports with a ball,NaN,523M,India,11.7M,14.7M
2,3,neymarjr,NJ 🇧🇷,Sports with a ball,NaN,198.9M,Brazil,6.7M,9.5M
3,4,kyliejenner,Kylie 🤍,Fashion,Modeling,376.3M,United States,2.7M,4.3M
4,5,kendalljenner,Kendall,Modeling,Fashion,268M,United States,3.2M,5M
...,...,...,...,...,...,...,...,...,...
995,996,thiago6,Thiago Alcantara,Sports with a ball,NaN,10.4M,United Kingdom,129.9K,222.7K
996,997,soycintiacossio,Cintia Cossio,Modeling,Adult content,6M,Colombia,313.4K,390.6K
997,998,feat.dino,이찬,Lifestyle,NaN,2.7M,South Korea,716.6K,876.4K
998,999,djokernole,Novak Djokovic,Sports with a ball,NaN,11.8M,United States,158.6K,197.5K


### TikTok:

In [7]:
tt_june= pd.read_csv('DATA/tiktokjune2022.csv')
tt_june

,Tiktoker,influencer name,Subscribers count,Views avg.,Likes avg,Comments avg.,Shares avg
0,___nicoichi___,NICO【平成フラミンゴ】,1.3M,1.9M,225.3K,1.7K,1.1K
1,_.anime.edits_0,video,97K,1.7M,75.6K,1.5K,8.7K
2,_.respect.boss._,Respect compilation 🤝🥶,1.5M,4.4M,241.9K,249,797
3,_.the.meme.hub._,RANDOM MEMES,2.3M,3.8M,787.7K,2.9K,9K
4,_andrewcurtiss,Andrew Curtis,2.5M,1.4M,300.4K,5.8K,4K
...,...,...,...,...,...,...,...
1002,zeth,Zeth,12.5M,6.7M,1.3M,3K,7.7K
1003,zodiac.boyfriend,Zodiac Boyfriend🪐🔮,1.8M,1.9M,272.7K,6.8K,3.6K
1004,zowloficial,Lord Zowl,1.1M,1.4M,178.1K,1.4K,2.1K
1005,zrt.foot,ZR7,100.8K,2.3M,218K,1.7K,6.4K


In [9]:
tt_sep= pd.read_csv('DATA/Tiktoksep2022.csv')
tt_sep

,S.no,Tiktoker name,Tiktok name,Subscribers,Views avg.,Likes avg.,Comments avg.,Shares avg.
0,1,jypestraykids,Stray Kids,13.8M,6.4M,2.3M,50.2K,34.2K
1,2,khaby.lame,Khabane lame,149.2M,17.3M,2.3M,15.2K,8.7K
2,3,scarlettsspam2,scarlett,2.1M,17.9M,845.8K,53.9K,6.3K
3,4,addisonre,Addison Rae,88.7M,22M,906.6K,7.6K,26.2K
4,5,belindatok,Belinda,4.8M,14.2M,1.5M,14.5K,15.3K
...,...,...,...,...,...,...,...,...
995,996,brendadialoy,Brenda Dialoy,371.9K,1.2M,187.5K,626,2K
996,997,jujufitcats,Juju Fitcats,4.3M,2.2M,280.8K,380,558
997,998,xoteam,XO Team,37.8M,2.5M,180.2K,859,450
998,999,kimsnwuo,️️,100.9K,957.4K,226.8K,908,1.7K


In [11]:
tt_nov= pd.read_csv('DATA/tiktoknov2022.csv')
tt_nov

,row-cell,Tiktoker name,Tiktok name,\nFollowers,Views (Avg.),Likes (Avg.),Comments (Avg.),Shares (Avg.)
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,yossikramer,Yossi,145.6K,26.8M,2.2M,30.1K,71.2K
2,2.0,mrbeast,MrBeast,55.6M,38.1M,3.8M,19K,7.8K
3,3.0,adinross,adin,5.4M,16.6M,2M,10.9K,12.5K
4,4.0,daniel.labelle,Daniel LaBelle,27.9M,11.8M,1.7M,6.9K,29.3K
...,...,...,...,...,...,...,...,...
996,996.0,nathanmccarron0,Nathan’s Art,1.6M,1.1M,117.6K,1.4K,1.1K
997,997.0,lacha21.1,Lazaro Gonzalez,669.4K,750.3K,118.3K,655,3K
998,998.0,hyoga.x,Hyoga,552K,1M,121.9K,1.9K,727
999,999.0,rrqr7tatsumaki,Rivaldi Fataah,1.1M,2M,112.5K,1K,414


In [12]:
tt_dec= pd.read_csv('DATA/TIKTOKDEC2022.csv')
tt_dec

,Rank,Tiktoker name,Tiktok name,followers,views(avg),likes(avg.),comments(avg.),shares(avg.)
0,1,mrbeast,MrBeast,60.3M,29.2M,3.5M,30.8K,7.2K
1,2,karolg,Karol G,42.4M,23.7M,3.4M,21.7K,25.7K
2,3,yzn47,يزن الأسمر,8.9M,48.9M,998.4K,16.3K,60.9K
3,4,centralcee,CentralCee,4.4M,19.8M,3.6M,23.3K,24.2K
4,5,adinross,adin,6.1M,21.1M,3.3M,17.5K,25.3K
...,...,...,...,...,...,...,...,...
995,996,jiembasands,Jiemba Sands,4.9M,3.4M,247.4K,207,264
996,997,crissa_ace,Crissa Jackson,14.9M,1.6M,141.7K,1.2K,580
997,998,ichadude,Alyssa & Dude,468.8K,2.3M,85.5K,997,1K
998,999,kanebrown,Kane Brown,5.2M,1.7M,96.7K,1.3K,1.2K


### Youtube:

In [13]:
y_june= pd.read_csv('DATA/youtubejune2022.csv')
y_june

,channel name,youTuber,Category,Category_2,Subscribers count,Country,Views avg.,Likes avg,Comments avg.
0,juptube,Galinha Pintadinha,Music & Dance,NaN,30.5M,-,0,N/A',N/A'
1,"Baby Big Mouth Kids - Sing, Dance, Explore","Baby Big Mouth Kids - Sing, Dance, Explore",Animation,Toys,10.6M,-,348.5,3,N/A'
2,shfa show India,shfa show India,Daily vlogs,NaN,18.9M,-,1.2M,8.7K,N/A'
3,Blippi - Educational Videos for Kids,Blippi - Educational Videos for Kids,Education,Toys,15.7M,-,1.2M,4.2K,N/A'
4,infobellstamil,infobells - Tamil,Education,NaN,26.5M,-,1.3M,5.8K,N/A'
...,...,...,...,...,...,...,...,...,...
1016,Power Vision,Power Vision,NaN,NaN,9.5M,United States,137.3K,1.3K,39
1017,POPSVIETNAM,POPS MUSIC,Music & Dance,NaN,9.5M,United States,4.6K,144,5
1018,Niki and Gabi,Niki and Gabi,Fashion,Humor,9.5M,United States,387.6K,17.8K,516
1019,Corridor,Corridor,Animation,NaN,9.5M,United States,1.9M,141.6K,4.7K


In [14]:
y_sep= pd.read_csv('DATA/Youtubesep2022.csv')
y_sep

,S.no,Name,Youtuber,Country,Subscribers,Category_2,Avg. views\r\n,Avg. likes,Avg Comments,Category_3
0,1,T-Series,tseries,India,224.4M,Music & Dance,141.5K,4.6K,125,NaN
1,2,Cocomelon - Nursery Rhymes,checkgate,NaN,142.5M,Education,12.5M,64.8K,0,NaN
2,3,PewDiePie,PewDiePie,United States,111.6M,Animation,1.9M,130.5K,7.4K,NaN
3,4,MrBeast,MrBeast6000,United States,104.1M,Video games,44.8M,2.2M,70.9K,Humor
4,5,✿ Kids Diana Show,✿ Kids Diana Show,NaN,100.5M,Animation,6.9M,34.8K,0,Toys
...,...,...,...,...,...,...,...,...,...,...
1042,996,Raditya Dika,Raditya Dika,Indonesia,9.9M,Food & Drinks,430.5K,14.8K,741,NaN
1043,997,Ini Talk Show,IniTalkShow,Indonesia,9.9M,Music & Dance,25.2K,290,17,Humor
1044,998,SpriteDer SPD,SpriteDer SPD,Thailand,9.9M,Humor,4.4M,154.4K,3.6K,NaN
1045,999,Classic Mr Bean,Classic Mr Bean,India,9.9M,NaN,132.6K,1.1K,53,NaN


In [15]:
y_nov= pd.read_csv('DATA/youtubenov2022.csv')
y_nov

,s.no,Youtube channel,youtuber name,Category,Followers,Country,\nViews (Avg.),Likes (Avg.),Comments (Avg.),Category-2
0,1,tseries,T-Series,Music & Dance,229.5M,India,103.8K,2.8K,158,NaN
1,2,checkgate,Cocomelon - Nursery Rhymes,Education,146.6M,NaN,9.5M,48.3K,0,NaN
2,3,PewDiePie,PewDiePie,Animation,111.6M,United States,1.4M,110.7K,7.8K,NaN
3,4,MrBeast6000,MrBeast,Video games,109.1M,United States,38.4M,3.1M,56.7K,Humor
4,5,✿ Kids Diana Show,✿ Kids Diana Show,Animation,102.5M,NaN,2.4M,15.7K,0,Toys
...,...,...,...,...,...,...,...,...,...,...
995,996,RUOK,RUOK,NaN,10.1M,Brazil,215.4K,25.2K,1.3K,NaN
996,997,FoxNewsChannel,Fox News,News & Politics,10.1M,United States,98.4K,4K,884,NaN
997,998,SULE PRODUCTIONS,SULE PRODUCTIONS,NaN,10.1M,Indonesia,218.3K,4.5K,306,NaN
998,999,MBCkpop,MBCkpop,Music & Dance,10.1M,Malaysia,19.7K,2.6K,92,NaN


In [16]:
y_dec= pd.read_csv('DATA/YOUTUBEDEC2022.csv')
y_dec

,s.no,Youtube channel,youtuber name,Category,Followers,Country,\nViews (Avg.),Likes (Avg.),Comments (Avg.),Category-2
0,1,tseries,T-Series,Music & Dance,229.5M,India,103.8K,2.8K,158,NaN
1,2,setindia,SET India,NaN,147.2M,India,0,0,0,NaN
2,3,checkgate,Cocomelon - Nursery Rhymes,Education,146.6M,NaN,9.5M,48.3K,0,NaN
3,4,MrBeast,MrBeast,Video games,117M,United States,68.5M,2.9M,99.9K,Humor
4,5,PewDiePie,PewDiePie,Animation,111.6M,United States,1.4M,110.7K,7.8K,NaN
...,...,...,...,...,...,...,...,...,...,...
998,999,ClassicMrBean,Classic Mr Bean,NaN,10.1M,India,48.3K,624,23,NaN
999,1000,Anupam Movie Songs,Anupam Movie Songs,Music & Dance,10.1M,Bangladesh,41.2K,315,18,NaN
1000,970,AnandAudio,Anand Audio,NaN,10.3M,India,175.5K,2.2K,109,NaN
1001,971,Wylsacom,Wylsacom,Science & Technology,10.3M,Russia,291.7K,13K,950,News & Politics


Cargar los 12 archivos que en el nombre tienen un mes, en 3  DataFrames, uno de cada red social, donde cada uno de los cuales debe contener las siguientes columnas:

Instagram:
* Username
* Name
* Subscribers o Followers
* Country
* Authentic
* Engagement
* Category1
* Category2
* Month

In [57]:
ig_months = ig_june
ig_months

,instagram name,influencer name,Category_1,Category_2,Subscribers count,Views avg.,Likes avg,Comments avg.
0,433,433,Sports with a ball,NaN,50.7M,Spain,274.9K,456.7K
1,____kimwoobin,김우빈,Lifestyle,NaN,3.6M,Indonesia,449.4K,592.7K
2,__youngbae__,TAEYANG,Music,NaN,13.4M,Indonesia,269.6K,304.1K
3,_agentgirl_,НАСТЯ ИВЛЕЕВА,Shows,NaN,18.6M,Russia,164.4K,189.4K
4,_hakkencoser_,Hakken 八犬,Lifestyle,Photography,3.5M,United States,525.1K,643K
...,...,...,...,...,...,...,...,...
1017,zidane,zidane,Sports with a ball,NaN,33M,Spain,412.4K,560.8K
1018,zidane,zidane,Sports with a ball,NaN,33M,Spain,412.4K,560.8K
1019,zkdlin,KAI,Music,NaN,14.1M,Indonesia,1.4M,1.7M
1020,zo,Lonzo Ball,Sports with a ball,NaN,14M,United States,145.8K,262K


In [61]:
ig_months = ig_months.rename(columns={'instagram name': 'Username',
                                      'influencer name': 'Name',
                                      'Subscribers count': 'Subscribers o Followers',
                                      'Views avg.': 'Country',
                                      #'': 'Authentic', #?
                                      #'': 'Engagement', #comentarios y likes?
                                      'Category_1': 'Category1',
                                      'Category_2': 'Category2',
                                     })
ig_months

,Username,influencer name,Category1,Category2,Subscribers o Followers,Country,Likes avg,Comments avg.
0,433,433,Sports with a ball,NaN,50.7M,Spain,274.9K,456.7K
1,____kimwoobin,김우빈,Lifestyle,NaN,3.6M,Indonesia,449.4K,592.7K
2,__youngbae__,TAEYANG,Music,NaN,13.4M,Indonesia,269.6K,304.1K
3,_agentgirl_,НАСТЯ ИВЛЕЕВА,Shows,NaN,18.6M,Russia,164.4K,189.4K
4,_hakkencoser_,Hakken 八犬,Lifestyle,Photography,3.5M,United States,525.1K,643K
...,...,...,...,...,...,...,...,...
1017,zidane,zidane,Sports with a ball,NaN,33M,Spain,412.4K,560.8K
1018,zidane,zidane,Sports with a ball,NaN,33M,Spain,412.4K,560.8K
1019,zkdlin,KAI,Music,NaN,14.1M,Indonesia,1.4M,1.7M
1020,zo,Lonzo Ball,Sports with a ball,NaN,14M,United States,145.8K,262K


In [60]:
ig_months.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1022 entries, 0 to 1021
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   Username                 1022 non-null   object
 1   influencer name          996 non-null    object
 2   Category1                927 non-null    object
 3   Category2                298 non-null    object
 4   Subscribers o Followers  1022 non-null   object
 5   Country                  1022 non-null   object
 6   Likes avg                1022 non-null   object
 7   Comments avg.            1006 non-null   object
dtypes: object(8)
memory usage: 64.0+ KB


TikTok:
* Username
* Name
* Followers o Suscribers
* Views
* Likes
* Comments
* Shares
* Month

Youtube:
* Username
* Name
* Country
* Suscribers o Folowers
* Views
* Likes
* Comments
* Category1
* Category2
* Month